### Initialize notebook

In [15]:
# Torch imports
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset
import PIL.Image as IMG

# Other imports
import matplotlib.pyplot as plt
import numpy as np
import os


# NOTE: Aashis, you should use the training images to develop your algorithm.
# The test images should be reserved for validating its performance afterwards
# os.chdir('/home/ak/Spring2018/ature')
os.chdir('/home/akhanal1/Spring2018/ature')

sep = os.sep

# Define folders (create them if needed)
Dirs = {}
Dirs['train_data']      = 'data'+sep+'DRIVE'+sep+'training'+sep +'patches'
Dirs['test_data']      = 'data'+sep+'DRIVE'+sep+'test'+sep +'patches'

# Set up execution flags
Flags = {}
Flags['useGPU'] = False

In [16]:
class DriveDataset(Dataset):
    
    ## INPUT
    # data_path should contain pickled numpy array of dimension N * (D+1)
    # Where extra one dimension stores the correct lable among (0, 1, 2, 3)
    
    ## self.labels contains one-hot encoding
    ## self.target contains true labels (single value)
    
    def __init__(self, data_path=None, height=None, width=None, transform=None, load_one=False):
        
        self.data = None
        self.height = height
        self.width = width
        self.transform = transform
        
        for data_file in os.listdir(data_path):
            
            data_file = os.path.join(data_path, data_file)
            print('Data file: ' + data_file)
            if self.data is None:
                self.data = np.load(data_file)
            else:
                self.data = np.concatenate((self.data, np.load(data_file)), axis=0)
            
            if load_one:
                break
    
        self.data_len = self.data.shape[0]
        
        self.labels = np.zeros((self.data_len, 4))
        self.target = self.data[:, self.height * self.width] 
        self.labels[range(self.data_len), self.target] = 1
        
        self.data = self.data[:,0:self.height * self.width]
        
        
    def __getitem__(self, index):
        
        a_data = self.data[index]
        img_arr = a_data.reshape(self.height, self.width)
        img = IMG.fromarray(img_arr)
        
        if self.transform is not None:
            img_tensor = self.transform(img)
#             
#         return (img_tensor, torch.LongTensor(self.target[index]))
        return (img_tensor, torch.from_numpy(self.labels[index])) 
    
    def __len__(self):
        return self.data_len

In [18]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = DriveDataset(data_path=Dirs['train_data'], height=31, width=31, transform=transform, load_one=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=2,
                                          shuffle=True, num_workers=1)
testset = DriveDataset(data_path=Dirs['test_data'], height=31, width=31, transform=transform, load_one=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=2,
                                          shuffle=True, num_workers=1)

classes = ('white', 'green', 'black', 'red')

Data file: data/DRIVE/training/patches/26_training.npy


In [21]:
IMG.fromarray(trainset.data[10000].reshape(31,31))

In [20]:
trainset.data.shape

(186895, 961)

### Define the network

In [23]:
class Net(nn.Module):
    def __init__(self, width):
        super(Net, self).__init__()
        nChannels = 1
        convKernelSize = 3
        convStride = 1
        mpKernelSize = 2
        mpStride = 2        
        padding = 0
        dilation = 1

        self.pool = nn.MaxPool2d(mpStride)

        self.conv1 = nn.Conv2d(nChannels, 6, convKernelSize)
        width = self.output_size(width,convKernelSize,padding,dilation,
            mpKernelSize,mpStride)
        self.conv2 = nn.Conv2d(6, 16, convKernelSize)
        width = self.output_size(width,convKernelSize,padding,dilation,
            mpKernelSize,mpStride)

        self.linearWidth = 16*int(width)*int(width)
        self.fc1 = nn.Linear(self.linearWidth, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.linearWidth)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    # Output size of a convolution, followed by max pooling operation
    def output_size(self,inputSize,convKernelSize,padding,dilation,
                    kernelSize,stride):
        return np.floor(((inputSize - 2*np.floor(convKernelSize/2)) + 
                        2*padding - dilation*(kernelSize-1) - 1) / stride + 1)

width = 31
net = Net(width)

# Send network to the GPU, if requested
if Flags['useGPU']:
    net.cuda()

# Define loss criterion
# criterion = nn.L1Loss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


### Train the network

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        labels = labels.long()
        labels = torch.max(labels,1)[1]


        if Flags['useGPU']:
            # Wrap them in Variable (CPU version)
            inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())                
        else:                
            # Wrap them in Variable (GPU version)
            inputs, labels = Variable(inputs), Variable(labels)
#         print(inputs, labels)
#         break

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.742
[1,  4000] loss: 0.699
[1,  6000] loss: 0.701
[1,  8000] loss: 0.661
[1, 10000] loss: 0.684
[1, 12000] loss: 0.676
[1, 14000] loss: 0.717
[1, 16000] loss: 0.708
[1, 18000] loss: 0.666
[1, 20000] loss: 0.671
[1, 22000] loss: 0.666
[1, 24000] loss: 0.679
[1, 26000] loss: 0.673
[1, 28000] loss: 0.670
[1, 30000] loss: 0.665
[1, 32000] loss: 0.530
[1, 34000] loss: 0.506
[1, 36000] loss: 0.466
[1, 38000] loss: 0.470
[1, 40000] loss: 0.423
[1, 42000] loss: 0.413
[1, 44000] loss: 0.398
[1, 46000] loss: 0.375
[1, 48000] loss: 0.352
[1, 50000] loss: 0.392
[1, 52000] loss: 0.366
[1, 54000] loss: 0.357
[1, 56000] loss: 0.349
[1, 58000] loss: 0.351
[1, 60000] loss: 0.364
[1, 62000] loss: 0.363
[1, 64000] loss: 0.328
[1, 66000] loss: 0.325
[1, 68000] loss: 0.329
[1, 70000] loss: 0.307
[1, 72000] loss: 0.316
[1, 74000] loss: 0.329
[1, 76000] loss: 0.323
[1, 78000] loss: 0.322
[1, 80000] loss: 0.306
[1, 82000] loss: 0.315
[1, 84000] loss: 0.319
[1, 86000] loss: 0.313
[1, 88000] 

### Test the network

In [8]:
correct = 0
total = 0
for data in testloader:
    images, labels = data        
    if Flags['useGPU']:
        images = images.cuda()
        labels = labels.cuda()

    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


NameError: name 'testloader' is not defined

### Check per-class performance

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
for data in testloader:
    images, labels = data
    if Flags['useGPU']:
        images = images.cuda()
        labels = labels.cuda()

    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


In [4]:
trainset.data.shape

(169795, 2601)